## Finite Markov Decision Processes
MDPs are a classical formalization of sequential decision making, where actions influence not just immediate rewards, but also subsequent situations, or states, and through those future rewards. It involves evaluative feedback similar to armed bandits but also an associative aspect which involves different actions in different situations.
<img src="markdown_images/agent-environment.png">
<h3 align = center> Agent-Environment Interface</h3>

S0;A0;R1; S1;A1;R2; S2;A2;R3; ……. 
In a finite MDP, the sets of states, actions, and rewards (S,A, and R) all have a finite number of elements. In this case, the random variables Rt and St have well defined discrete probability distributions dependent only on the preceding state and action.

**Policies and Value functions**

Almost all reinforcement learning algorithms involve estimating value functions -- functions of states (or of state-action pairs) that estimate how good it is for the agent to be in a given state (or how good it is to perform a given action in a given state). The notion of  “how good" here is defined in terms of future rewards that can be expected, or, to be precise, in terms of expected return. Of course the rewards the agent can expect to receive in the future depend on what actions it will take. Accordingly, value functions are defined with respect to particular ways of acting, called policies.

**state-value function for policy 𝜋** :: Expected reward from state s and following policy  𝜋 afterwards.
<img src="markdown_images/state-value.png">

**action-value function for policy  𝜋**  :: Expected reward by taking action a from state s and following  𝜋 thereafter.
<img src="markdown_images/action-value.png">	

**Bellman equation for state value**
Basically a walk from current state using the policy and the average reward it receives.

<img src="markdown_images/bellman-state.png">

**Bellman equation for action value**

<img src="markdown_images/bellman-action.png">

**Optimal Policies and Optimal Value functions**

<img src="markdown_images/optimal-policy.png">

### Iterative Policy Evaluation

Calculates the state-value function V(s) for a given policy. In DP this is done using a "full backup". At each state, we look ahead one step at each possible action and next state. We can only do this because we have a perfect model of the environment.


<img src="markdown_images/policy-eval.png">

### Grid World Problem
The cells of the grid correspond to the states of the environment. At each cell, four actions
are possible: north, south, east, and west, which deterministically cause the agent to move one cell
in the respective direction on the grid. Actions that would take the agent off the grid leave its location
unchanged, but also result in a reward of -1. Other actions result in a reward of 0, except those that
move the agent out of the special states A and B. From state A, all four actions yield a reward of +10
and take the agent to A'. From state B, all actions yield a reward of +5 and take the agent to B'.
<img src="markdown_images/gridworld.png">


##### Policy evaluation for equiprobable policy

In [3]:
import numpy as np
from tabulate import tabulate
"""
    0 - left(-1,0)
    1 - right(1,0)
    2 - down(0,1)
    3 - up(0,-1)
"""
directions = [(-1,0),(1,0),(0,1),(0,-1)]
state_A = (1,0)
state_Aprime = (1,4)
state_B = (3,0)
state_Bprime = (3,2)
gamma = 0.9 

def step(state,action):
    direction = directions[action]
    if state == state_A:
        return state_Aprime,10
    if state == state_B:
        return state_Bprime,5
    next_state = (state[0] + direction[0] , state[1] + direction[1])
    if next_state[0] <0 or next_state[0]>=5 or next_state[1] <0 or next_state[1]>=5:
        return state,-1
    else :
        return next_state,0

value = np.zeros((5,5),dtype=float)

while True:
    delta = 0
    for i in range(5):
        for j in range(5):
            new_value = 0
            for action in range(4):
                (next_i, next_j), reward = step((i, j), action)
                # bellman equation
                new_value +=  0.25 * (reward + gamma * value[next_i, next_j])
            delta = max(delta,np.abs(value[i,j]-new_value))
            value[i,j] = new_value
    if delta < 1e-2:
        break
table = tabulate(value.transpose(), tablefmt="fancy_grid")
print("Policy evaluation for equiprobable policy ::")
print(table)

Policy evaluation for equiprobable policy ::
╒════════════╤═══════════╤═══════════╤═══════════╤═══════════╕
│  3.36249   │  8.83466  │  4.4715   │  5.36318  │  1.53396  │
├────────────┼───────────┼───────────┼───────────┼───────────┤
│  1.571     │  3.03591  │  2.29086  │  1.94644  │  0.585912 │
├────────────┼───────────┼───────────┼───────────┼───────────┤
│  0.0987228 │  0.780693 │  0.712606 │  0.396028 │ -0.365913 │
├────────────┼───────────┼───────────┼───────────┼───────────┤
│ -0.926015  │ -0.393321 │ -0.315778 │ -0.548119 │ -1.14625  │
├────────────┼───────────┼───────────┼───────────┼───────────┤
│ -1.81013   │ -1.30313  │ -1.19026  │ -1.38553  │ -1.93847  │
╘════════════╧═══════════╧═══════════╧═══════════╧═══════════╛


### Policy Iteration
Each policy is guaranteed to be a strict improvement over the previous one (unless it is already optimal). Because a finite MDP has only a nite number of policies, this process must converge to an optimal policy and optimal value function in a nite number of iterations.
<img src="markdown_images/gridworld_new.png">

In [30]:
import numpy as np
from tabulate import tabulate
"""
    0 - left(-1,0)
    1 - right(1,0)
    2 - down(0,1)
    3 - up(0,-1)
"""
directions = [(-1,0),(1,0),(0,1),(0,-1)]
gamma = 0.9 
def step(state,action):
    direction = directions[action]
    if state == state_A:
        return state_Aprime,10
    if state == state_B:
        return state_Bprime,5
    next_state = (state[0] + direction[0] , state[1] + direction[1])
    if next_state[0] <0 or next_state[0]>=5 or next_state[1] <0 or next_state[1]>=5:
        return state,-1
    else :
        return next_state,0

def policy_evaluation(policy,value):
    while True:
        delta = 0
        for i in range(5):
            for j in range(5):
                new_value = 0
                for action,prob in enumerate(policy[i][j]):
                    (next_i, next_j), reward = step((i, j), action)
                    # bellman equation
                    new_value +=  prob * (reward + gamma * value[next_i, next_j])
                delta = max(delta,np.abs(value[i,j]-new_value))
                value[i,j] = new_value
        if delta < 1e-3:
            return value

def get_q_values(i,j,value,policy):
    Q_values = np.zeros(4)
    for a in range(4):
        (next_i, next_j), reward = step((i, j), a)
        Q_values[a] += reward + gamma * value[next_i, next_j]
    return Q_values

def policy_iteration():
    value = np.zeros((5,5))
    policy = np.ones((5,5,4))/4
    while True:
        policy_stable = True
        value = policy_evaluation(policy,value)
        for i in range(5):
            for j in range(5):
                a_old_max = np.random.choice(np.flatnonzero(policy[i][j] == policy[i][j].max()))
                Q_values = get_q_values(i,j,value,policy)
                if a_old_max != np.argmax(Q_values):
                    policy_stable = False
                policy[i][j] = np.zeros(4)
                policy[i][j][np.argmax(Q_values)] = 1
        if policy_stable :
            return policy,value
policy,value = policy_iteration()
table = tabulate(value.transpose(), tablefmt="fancy_grid")
print("Policy evaluation for Best policy ::")
print(table)
print(np.argmax(policy,axis=2).transpose())

Policy evaluation for Best policy ::
╒═════════╤═════════╤═════════╤═════════╤═════════╕
│ 21.9775 │ 24.4194 │ 21.9775 │ 19.4194 │ 17.4775 │
├─────────┼─────────┼─────────┼─────────┼─────────┤
│ 19.7797 │ 21.9775 │ 19.7797 │ 17.8018 │ 16.0216 │
├─────────┼─────────┼─────────┼─────────┼─────────┤
│ 17.8018 │ 19.7797 │ 17.8018 │ 16.0216 │ 14.4194 │
├─────────┼─────────┼─────────┼─────────┼─────────┤
│ 16.0216 │ 17.8018 │ 16.0216 │ 14.4194 │ 12.9775 │
├─────────┼─────────┼─────────┼─────────┼─────────┤
│ 14.4194 │ 16.0216 │ 14.4194 │ 12.9775 │ 11.6797 │
╘═════════╧═════════╧═════════╧═════════╧═════════╛
[[1 0 0 0 0]
 [1 3 0 0 0]
 [1 3 0 0 0]
 [1 3 0 0 0]
 [1 3 0 0 0]]
